In [349]:
import aoc
import numpy as np

%reload_ext autoreload

day = 14
sample = False
sample_number = 1

In [350]:
input_list = aoc.split_contents(aoc.read_input(f'Input/day_{day:02}{"_sample"+str(sample_number) if sample else ""}.txt'))

# Part 1

In [351]:
def pretty_print(array):
    # Prints the array with the bottom left corner being 0,0
    array_t = array.T
    for y in reversed(range(array_t.shape[1])):
        print(''.join(array_t[y]))


def get_cube_rock_dict(array):
    """
    Builds a dictionary of all the cube rocks (#s) on the platform. The dictionary key is a tuple
    of the x,y coordinates of the cube rock. The value of the key, initialized to zero, will be the
    number of rolling rocks stacked up against it based on the tilt direction.
    :param array: Input platform
    :type array: NumPy 2d array
    :return: Dictionary of cube rocks
    :rtype: Dictionary
    """
    cube_rock_dict = dict()
    for x in range(array.shape[0]):
        for y in range(array.shape[1]):
            if array[x][y] == '#':
                cube_rock_dict[(x,y)] = 0
    # build a pseudo row of rocks around the edges
    for x in range(array.shape[0]):
        cube_rock_dict[(x,array.shape[1])] = 0
        cube_rock_dict[(x,-1)] = 0
    for y in range(array.shape[1]):
        cube_rock_dict[(array.shape[0]),y] = 0
        cube_rock_dict[(-1,y)] = 0
    return cube_rock_dict


def count_rolling_rocks_between(coord1, coord2, array):
    """
    A count of rolling rocks between any 2 vertically or horizontally aligned coordinates
    :param coord1: x,y coordinates of point 1
    :type coord1: Tuple
    :param coord2: x,y coordinates of point 2
    :type coord2: Tuple
    :param array: 2d platform
    :type array: NumPy 2d array
    :return: Count of rolling rocks 'O' between the 2 coordinates
    :rtype: Integer
    """
    if coord1[0] == coord2[0]:
        # The x coordinates are the same so this is a vertical stack
        column = coord1[0]
        row_start = min(coord1[1],coord2[1])
        row_end = max(coord1[1],coord2[1])
        stack = array[column,row_start+1:row_end]
    elif coord1[1] == coord2[1]:
        # The y coordinates are the same, to this is a horizontal stack
        row = coord1[1]
        col_start = min(coord1[0],coord2[0])
        col_end = max(coord1[0],coord2[0])
        stack = array[col_start+1:col_end,row]
    else:
        raise ValueError(f'The coordinates {coord1} and {coord2} are not orthogonal.')
    # counts all the characters in the array and zips into a dict
    unique,counts = np.unique(stack, return_counts=True)
    stack_dict = dict(zip(unique,counts))
    # just return the count of O's
    return 0 if stack_dict.get('O') is None else stack_dict.get('O')


def get_next_cube_coordinate(coord, direction, array):
    """
    General function to find the coordinates of the next cube rock looking in any direction
    from the given coordinate. If there is no cube rock between the given coordinate and the edge,
    then just return the edge coordinate in the given direction.
    :param coord: x,y coordinate to look from
    :type coord: Tuple
    :param direction: Compass direction (NSEW) to look in
    :type direction: String
    :param array: 2d NumPy array of the platform
    :type array: NumPy 2d array
    :return: x,y coordinates of the next cube rock '#'
    :rtype: Tuple
    """
    if direction not in ['N','S','E','W']:
        raise ValueError(f'The direction needs to be in NSEW. You provided {direction}')
    if direction in ['N','S']:
        if direction == 'S':
            aoc.logger.info(f'{coord[0]=} {coord[1]=} {array.shape[1]=}')
            vector = array[coord[0],::-1][array.shape[1]  - coord[1]:]
            aoc.logger.info(f'{vector=}')
            try:
                cube_coord = (coord[0], coord[1] - list(vector).index('#') - 1)
            except ValueError:
                # No cubes to the south, so just return the edge coordinate.
                aoc.logger.info(f'No cubes to the south')
                cube_coord = (coord[0],-1)
        else:
            # TODO: Needs fixing
            vector = array[coord[0],:][coord[1]+1:]
            aoc.logger.info(f'{vector=}')
            try:
                cube_coord = (coord[0], coord[1] + list(vector).index('#') +1)
            except ValueError:
                # No cubes to the north, so just return the edge coordinate.
                aoc.logger.info(f'No cubes to the north')
                cube_coord = (coord[0],array.shape[1])
    else:
        if direction == 'E':
            # TODO: Needs fixing
            vector = array[:,coord[1]][coord[0]+1:]
            try:
                cube_coord = (coord[0] + list(vector).index('#') + 1, coord[1])
            except ValueError:
                # No cubes to the east, so just return the edge coordinate.
                cube_coord = (array.shape[0]-1,coord[1])
        else:
            # TODO: Needs fixing
            vector = array[::-1,coord[1]][array.shape[0] - coord[0]:]
            try:
                cube_coord = (coord[0] - list(vector).index('#') -1, coord[1])
            except ValueError:
                # No cubes to the west, so just return the edge coordinate.
                cube_coord = (0, coord[1])
    return cube_coord

In [352]:
# Generate the platform as a 2d NumPy array, bottom left is 0,0
# The y-axis increases up
# The x-axis increases right
row_list = list()
for i,input_row in enumerate(input_list):
    row_list.append([x for x in input_row])
platform = np.array(row_list[::-1]).T

In [353]:
aoc.logger.setLevel(30)
tilt_direction = 'N'
limit_dict = {
    'N': {'x_bounds':(0,platform.shape[0]-1), 'y_bounds': (0,platform.shape[1])},
    'S': {'x_bounds':(0,platform.shape[0]-1), 'y_bounds': (-1,platform.shape[1]-1)},
    'E': {'x_bounds':(0,platform.shape[0]), 'y_bounds': (0,platform.shape[1]-1)},
    'W': {'x_bounds':(-1,platform.shape[0]-1), 'y_bounds': (0,platform.shape[1]-1)}
}
cube_rocks = get_cube_rock_dict(platform)
for item in cube_rocks:
    if ((limit_dict[tilt_direction]['x_bounds'][0] <= item[0] <= limit_dict[tilt_direction]['x_bounds'][1]) and
        (limit_dict[tilt_direction]['y_bounds'][0] <= item[1] <= limit_dict[tilt_direction]['y_bounds'][1])):
        next_cube_coord = get_next_cube_coordinate(item, 'S', platform)
        rolling_rock_count = count_rolling_rocks_between(item,next_cube_coord,platform)
        cube_rocks[item] = rolling_rock_count

In [354]:
aoc.logger.setLevel(30)
total_load = 0
for cube_rock in cube_rocks.items():
    if cube_rock[1] > 0:
        load = sum(range((cube_rock[0][1] - cube_rock[1] + 1), (cube_rock[0][1]) + 1))
        total_load += load
        aoc.logger.info(f'{cube_rock=} {load=}')

In [355]:
total_load

111339

# Part 2

In [356]:
limit_dict

{'N': {'x_bounds': (0, 99), 'y_bounds': (0, 100)},
 'S': {'x_bounds': (0, 99), 'y_bounds': (-1, 99)},
 'E': {'x_bounds': (0, 100), 'y_bounds': (0, 99)},
 'W': {'x_bounds': (-1, 99), 'y_bounds': (0, 99)}}